In [1]:
import torch.nn as nn
import librosa
import numpy as np
import torch
import torch.utils.data
import matplotlib.pyplot as plt
import IPython.display as ipd

In [2]:
# from google.colab import files
# files.upload()


In [3]:
# files.upload()

In [4]:
s, sr=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [5]:
s_abs = np.abs(S)
x_abs = np.abs(X)
s = torch.tensor(np.transpose(s_abs)).cuda()
x = torch.tensor(np.transpose(x_abs)).cuda()

In [6]:
print(s.shape)
print(x.shape)

torch.Size([2459, 513])
torch.Size([2459, 513])


In [7]:
BATCH = 128
train_loader = torch.utils.data.DataLoader(x, batch_size=BATCH)
test_loader = torch.utils.data.DataLoader(s, batch_size=BATCH)

In [8]:
test_iter = iter(train_loader)
test_data = test_iter.next()
print(test_data.shape)

torch.Size([128, 513])


In [9]:
my_data = test_data.squeeze().cuda()
print(my_data.shape)

torch.Size([128, 513])


In [10]:
def get_shape(input, filter, padding, stride):
  return (((input - filter + (2*padding)) / stride)) + 1

get_shape(511, 2,2, 1)

514.0

In [11]:
class NeuralNetwork(nn.Module):
  
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    
    conv1_channel_in = 1
    conv1_number_of_filter = 16
    conv1_kernel_size = 2
    conv1_stride = 1
    conv1_padding = 0
    
    max_poo1_kernel_size  = 2
    max_poo1_kernel_stride  = 1
    
    conv2_number_of_filter = 32
    conv2_kernel_size = 2
    conv2_stride = 1
    conv2_padding = 2
    
    max_poo2_kernel_size  = 2
    max_poo2_kernel_stride  = 1
    
   
    conv3_number_of_filter = 64
    conv3_kernel_size = 2
    conv3_stride = 1
    conv3_padding = 2
    
    max_poo3_kernel_size  = 2
    max_poo3_kernel_stride  = 1
    
    
    
    self.conv1 = nn.Sequential(
            nn.Conv2d(conv1_channel_in, conv1_number_of_filter, kernel_size=conv1_kernel_size, stride=conv1_stride, padding=conv1_padding),
            nn.ReLU(),       
            nn.MaxPool2d(kernel_size=max_poo1_kernel_size, stride=max_poo1_kernel_stride))
    
    conv1_firstdim = get_shape(513, conv1_kernel_size, conv1_padding, conv1_stride)
    conv1_seconddim = get_shape(20, conv1_kernel_size, conv1_padding, conv1_stride)
    
    print("convolution 1")
    print("x:", conv1_firstdim)
    print("y:", conv1_seconddim)
    
    pool1_firstdim = get_shape(conv1_firstdim, max_poo1_kernel_size, 0, max_poo1_kernel_stride)
    pool1_seconddim = get_shape(conv1_seconddim, max_poo1_kernel_size, 0, max_poo1_kernel_stride)
    
    print("pool 1")
    print("x:", pool1_firstdim)
    print("y:", pool1_seconddim)
    
    self.conv2 = nn.Sequential(
            nn.Conv2d(conv1_number_of_filter, conv2_number_of_filter, kernel_size=conv2_kernel_size, stride=conv2_stride,  padding=conv2_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=max_poo2_kernel_size, stride=max_poo2_kernel_stride))
    
    conv2_firstdim = get_shape(pool1_firstdim, conv2_kernel_size, conv2_padding, conv2_stride)
    conv2_seconddim = get_shape(pool1_seconddim, conv2_kernel_size, conv2_padding, conv2_stride)
    
    print("convolution 2")
    print("x:",  conv2_firstdim)
    print("y:", conv2_seconddim)
    
    pool2_firstdim = get_shape(conv2_firstdim, max_poo2_kernel_size, 0, max_poo2_kernel_stride)
    pool2_seconddim = get_shape(conv2_seconddim, max_poo2_kernel_size,0,  max_poo2_kernel_stride)
    
    print("pool 2")
    print("x:",  pool2_firstdim)
    print("y:", pool2_seconddim)
    
    self.conv3 = nn.Sequential(
            nn.Conv2d(conv2_number_of_filter, conv3_number_of_filter, kernel_size=conv3_kernel_size, stride=conv3_stride,  padding=conv3_padding),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=max_poo2_kernel_size, stride=max_poo2_kernel_stride))
    
    conv3_firstdim = get_shape(pool2_firstdim, conv3_kernel_size, conv3_padding,  conv3_stride)
    conv3_seconddim = get_shape(pool2_seconddim, conv3_kernel_size, conv3_padding,  conv3_stride)
    
    print("convolution 3")
    print("x:", conv3_firstdim)
    print("y:", conv3_seconddim)
    
    pool3_firstdim = get_shape(conv3_firstdim, max_poo3_kernel_size, 0, max_poo3_kernel_stride)
    pool3_seconddim = get_shape(conv3_seconddim, max_poo3_kernel_size, 0,  max_poo3_kernel_stride)
    
    print("Pool 3")
    print("x:", pool3_firstdim)
    print("y:", pool3_seconddim)
    
    final_out_dim = pool3_firstdim * pool3_seconddim * 64
    print(final_out_dim)
    self.fc1 = nn.Linear(in_features=int(final_out_dim), out_features=513)
    
  def forward(self, data):
#     print(data.shape)
    data = data[np.newaxis, np.newaxis,:, :]
    output = self.conv1(data)
#     print(data.shape)
    output = self.conv2(output)
#     print(output.shape)
    output = self.conv3(output)
    output = output.reshape(output.shape[0], -1)
#     print(output.shape)
    output = self.fc1(output)
    final = torch.nn.functional.relu(output)
    return final

In [12]:
neural_network = NeuralNetwork().cuda()
# child_counter = 0
# for child in neural_network.children():
#   print(" child", child_counter, "is:")
#   print(child)
#   for param in child.parameters():
#     print(param)
#   child_counter += 1

convolution 1
x: 512.0
y: 19.0
pool 1
x: 511.0
y: 18.0
convolution 2
x: 514.0
y: 21.0
pool 2
x: 513.0
y: 20.0
convolution 3
x: 516.0
y: 23.0
Pool 3
x: 515.0
y: 22.0
725120.0


In [13]:
# using Adam with learning rat .001 keeping everything else default.
# neural_network = NeuralNetwork().cuda()
loss_function= nn.MSELoss()
para = neural_network.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.01)
loss_list = []
epochs = 1
for i in range(epochs):
    input_iter = iter(train_loader)
    target_iter = iter(test_loader)
    while True:
        batch_count = 0 
        try:
            input_batch = input_iter.next()
#             print("input batch shape", input_batch.shape)
            target_batch = target_iter.next()
            batch_count = batch_count + 1
#             print("batch", str(batch_count))
            for j in range(input_batch.shape[0] - 20):
#                 print(j)
                input_set = input_batch[j:j+20,:].cuda()
                target_set = target_batch[j:j+20,:].cuda()
                network_output  = neural_network(input_set)
                loss = loss_function(network_output , target_set)
                loss.backward()
                optimizer.step()
                print(i)
        except StopIteration:
            break
    loss_list.append(loss.data.cpu().numpy())
# loss_list.append(loss.data.cpu().numpy())
print(loss.data)
plt.plot(range(epochs), loss_list)
plt.show()   

RuntimeError: CUDA out of memory. Tried to allocate 1.39 GiB (GPU 0; 6.00 GiB total capacity; 4.17 GiB already allocated; 442.45 MiB free; 78.00 KiB cached)